## A very basic RAG

You would never build a RAG system this basic. But it helps illustrate the problems we are trying to solve with some of the more advanced techniques.

In [1]:
#%pip install --quiet llama-index llama-index-retrievers-bm25 llama-index-llms-anthropic anthropic

In [2]:
MODEL_ID = "claude-3-7-haiku-latest"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"

In [3]:
# Configure logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## Utility: cache urls to local directory

In [4]:
import os
import re
import time
import hashlib
import requests
import shutil
from pathlib import Path
from typing import List, Optional, Dict, Union, Tuple, Any
from urllib.parse import urlparse

class CacheManager:
    """
    Manages the local cache for downloaded files.
    
    Attributes:
        cache_dir (Path): Path to the cache directory.
    """
    
    def __init__(self, cache_dir: str = "./.cache"):
        """
        Initialize the cache manager.
        
        Args:
            cache_dir (str): Path to the cache directory. Defaults to "./.cache".
        """
        self.cache_dir = Path(cache_dir)
        self._ensure_cache_dir()
    
    def _ensure_cache_dir(self) -> None:
        """Create the cache directory if it doesn't exist."""
        if not self.cache_dir.exists():
            self.cache_dir.mkdir(parents=True)
            logger.info(f"Created cache directory at {self.cache_dir}")
    
    def _get_cache_filename(self, url: str) -> str:
        """
        Generate a unique filename for a URL.
        
        Args:
            url (str): The URL to generate a filename for.
            
        Returns:
            str: A unique filename based on the URL.
        """
        # Extract the filename from the URL if possible
        parsed_url = urlparse(url)
        path_parts = parsed_url.path.split('/')
        original_filename = path_parts[-1] if path_parts[-1] else "index"
        
        # Create a hash of the URL to ensure uniqueness
        url_hash = hashlib.md5(url.encode()).hexdigest()[:10]
        
        # Combine original filename with hash
        if '.' in original_filename:
            name_parts = original_filename.split('.')
            extension = name_parts[-1]
            base_name = '.'.join(name_parts[:-1])
            return f"{base_name}_{url_hash}.{extension}"
        else:
            return f"{original_filename}_{url_hash}.txt"
    
    def get_cache_path(self, url: str) -> Path:
        """
        Get the cache path for a URL.
        
        Args:
            url (str): The URL to get the cache path for.
            
        Returns:
            Path: The path where the cached file would be stored.
        """
        filename = self._get_cache_filename(url)
        return self.cache_dir / filename
    
    def is_cached(self, url: str) -> bool:
        """
        Check if a URL is already cached.
        
        Args:
            url (str): The URL to check.
            
        Returns:
            bool: True if the URL is cached, False otherwise.
        """
        cache_path = self.get_cache_path(url)
        return cache_path.exists()
    
    def get_cached_content(self, url: str) -> Optional[str]:
        """
        Get the cached content for a URL.
        
        Args:
            url (str): The URL to get the cached content for.
            
        Returns:
            Optional[str]: The cached content if available, None otherwise.
        """
        if not self.is_cached(url):
            return None
        
        cache_path = self.get_cache_path(url)
        try:
            with open(cache_path, 'r', encoding='utf-8') as f:
                return f.read()
        except Exception as e:
            logger.warning(f"Error reading cached file for {url}: {e}")
            return None
    
    def cache_content(self, url: str, content: str) -> bool:
        """
        Cache content for a URL.
        
        Args:
            url (str): The URL the content was downloaded from.
            content (str): The content to cache.
            
        Returns:
            bool: True if caching was successful, False otherwise.
        """
        self._ensure_cache_dir()
        cache_path = self.get_cache_path(url)
        
        try:
            with open(cache_path, 'w', encoding='utf-8') as f:
                f.write(content)
            logger.info(f"Cached content for {url} at {cache_path}")
            return True
        except Exception as e:
            logger.error(f"Error caching content for {url}: {e}")
            return False
    
    def clear_cache(self) -> bool:
        """
        Clear all cached files.
        
        Returns:
            bool: True if clearing was successful, False otherwise.
        """
        try:
            if self.cache_dir.exists():
                for file_path in self.cache_dir.iterdir():
                    if file_path.is_file():
                        file_path.unlink()
                logger.info("Cache cleared successfully")
            return True
        except Exception as e:
            logger.error(f"Error clearing cache: {e}")
            return False
    
    def get_cache_size(self) -> Tuple[int, str]:
        """
        Get the total size of the cache.
        
        Returns:
            Tuple[int, str]: A tuple containing the size in bytes and a human-readable size.
        """
        total_size = 0
        
        if self.cache_dir.exists():
            for file_path in self.cache_dir.iterdir():
                if file_path.is_file():
                    total_size += file_path.stat().st_size
        
        # Convert to human-readable format
        units = ['B', 'KB', 'MB', 'GB']
        size_human = total_size
        unit_index = 0
        
        while size_human > 1024 and unit_index < len(units) - 1:
            size_human /= 1024
            unit_index += 1
        
        human_readable = f"{size_human:.2f} {units[unit_index]}"
        return total_size, human_readable
    
    def list_cached_files(self) -> List[Dict[str, Any]]:
        """
        List all cached files with metadata.
        
        Returns:
            List[Dict[str, Any]]: A list of dictionaries containing file information.
        """
        files_info = []
        
        if self.cache_dir.exists():
            for file_path in self.cache_dir.iterdir():
                if file_path.is_file():
                    stat = file_path.stat()
                    files_info.append({
                        'filename': file_path.name,
                        'path': str(file_path),
                        'size_bytes': stat.st_size,
                        'last_modified': time.ctime(stat.st_mtime)
                    })
        
        return files_info

## Step 1: Index document

We will break up the document as sentences, and index it using BM25
See: https://kmwllc.com/index.php/2020/03/20/understanding-tf-idf-and-bm-25/

In [5]:
from llama_index.core import Document, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.indices.document_summary import DocumentSummaryIndex
from llama_index.core.storage.docstore import SimpleDocumentStore

class GutenbergTextLoadError(Exception):
    """Exception raised for errors in loading Gutenberg text files."""
    pass


class GutenbergToBM25LlamaIndex:
    """
    A class to load text files from Project Gutenberg into LlamaIndex using BM25.
    
    This class handles fetching text content from URLs, processing Gutenberg-specific
    formatting, and creating a document store indexed by BM25.
    
    Attributes:
        cache_manager (CacheManager): Manager for the local cache.
        chunk_size (int): Size of text chunks for processing.
        chunk_overlap (int): Overlap between text chunks.
        docstore (SimpleDocumentStore): Document store for storing processed documents.
    """
    
    def __init__(
        self,
        cache_dir: str = "./.cache",
        chunk_size: int = 1024,
        chunk_overlap: int = 20
    ):
        """
        Initialize the GutenbergToBM25LlamaIndex loader.
        
        Args:
            chunk_size (int): Size of text chunks for processing. Defaults to 1024.
            chunk_overlap (int): Overlap between text chunks. Defaults to 20.
        """
        self.cache_manager = CacheManager(cache_dir)
        self.anthropic_api_key = os.environ.get("ANTHROPIC_API_KEY")
        
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.default_url = "https://www.gutenberg.org/files/53669/53669-0.txt"
        
        # Initialize a simple document store
        self.docstore = SimpleDocumentStore()
        
        Settings.node_parser = SentenceSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap
        )
        
        logger.info("GutenbergToBM25LlamaIndex initialized with Anthropic Haiku model")
    
    def fetch_text_from_url(self, url: str) -> str:
        """
        Fetch text content from a URL with caching
        
        Args:
            url (str): URL to fetch text from.
            
        Returns:
            str: Text content from the URL.
            
        Raises:
            GutenbergTextLoadError: If there's an error fetching or processing the URL.
        """
        if self.cache_manager.is_cached(url):
            logger.info(f"Loading {url} from cache")
            cached_content = self.cache_manager.get_cached_content(url)
            if cached_content:
                return cached_content
            logger.warning(f"Cached content for {url} could not be read, downloading again")
        
        try:
            logger.info(f"Fetching text from URL: {url}")
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            
            # Check if content is text
            content_type = response.headers.get('Content-Type', '')
            if 'text/plain' not in content_type and 'text/html' not in content_type:
                raise GutenbergTextLoadError(f"URL does not contain text content: {content_type}")
            
            # Detect encoding or use utf-8 as fallback
            encoding = response.encoding or 'utf-8'
            content = response.content.decode(encoding)
        
            # Cache the downloaded content
            self.cache_manager.cache_content(url, content)
            
            return content
        except requests.RequestException as e:
            raise GutenbergTextLoadError(f"Error fetching URL {url}: {str(e)}")
        except UnicodeDecodeError as e:
            raise GutenbergTextLoadError(f"Error decoding content from {url}: {str(e)}")
    
    def clean_gutenberg_text(self, text: str) -> str:
        """
        Clean Project Gutenberg text by removing headers, footers, and license information.
        
        Args:
            text (str): Raw text from Project Gutenberg.
            
        Returns:
            str: Cleaned text with Gutenberg-specific content removed.
        """
        # Pattern to find the start of the actual content (after header)
        start_markers = [
            r"\*\*\* START OF (THIS|THE) PROJECT GUTENBERG EBOOK .+? \*\*\*",
            r"\*\*\* START OF THE PROJECT GUTENBERG .+? \*\*\*",
            r"\*\*\*START OF THE PROJECT GUTENBERG EBOOK .+? \*\*\*",
            r"START OF (THIS|THE) PROJECT GUTENBERG EBOOK"
        ]
        
        # Pattern to find the end of the content (before footer)
        end_markers = [
            r"\*\*\* END OF (THIS|THE) PROJECT GUTENBERG EBOOK .+? \*\*\*",
            r"\*\*\* END OF THE PROJECT GUTENBERG .+? \*\*\*", 
            r"\*\*\*END OF THE PROJECT GUTENBERG EBOOK .+? \*\*\*",
            r"END OF (THIS|THE) PROJECT GUTENBERG EBOOK"
        ]
        
        # Find start of content
        start_pos = 0
        for marker in start_markers:
            match = re.search(marker, text, re.IGNORECASE)
            if match:
                start_pos = match.end()
                break
        
        # Find end of content
        end_pos = len(text)
        for marker in end_markers:
            match = re.search(marker, text, re.IGNORECASE)
            if match:
                end_pos = match.start()
                break
        
        # Extract and clean the content
        content = text[start_pos:end_pos].strip()
        
        # Remove extra whitespace
        content = re.sub(r'\n{3,}', '\n\n', content)
        
        logger.info(f"Cleaned Gutenberg text: removed {start_pos} chars from start, "
                   f"{len(text) - end_pos} chars from end")
        
        return content
    
    def load_from_url(self, url: Optional[str] = None):
        """
        Load text from a URL and create a BM25 retriever.
        
        Args:
            url (str, optional): URL to load text from. If None, uses the default URL.
            
        Returns:
            BM25Retriever: A BM25 retriever configured with the loaded documents.
            
        Raises:
            GutenbergTextLoadError: If there's an error loading or processing the text.
        """
        url = url or self.default_url
        
        try:
            # Fetch and clean the text
            raw_text = self.fetch_text_from_url(url)
            cleaned_text = self.clean_gutenberg_text(raw_text)
            
            # Create a document with metadata
            parsed_url = urlparse(url)
            filename = os.path.basename(parsed_url.path)
            
            document = Document(
                text=cleaned_text,
                metadata={
                    "source": url,
                    "filename": filename,
                    "date_loaded": time.strftime("%Y-%m-%d %H:%M:%S")
                }
            )
            
            # Parse the document into nodes
            nodes = Settings.node_parser.get_nodes_from_documents([document])
            
            # Add nodes to the document store
            self.docstore.add_documents(nodes)
 
            logger.info(f"Successfully loaded text from {url} -- {len(nodes)} nodes created.")
 
        except Exception as e:
            raise GutenbergTextLoadError(f"Error loading from URL {url}: {str(e)}")
            
    def get_indexed_documents(self) -> SimpleDocumentStore:
        return self.docstore

In [6]:
index = GutenbergToBM25LlamaIndex(chunk_size=100, chunk_overlap=20)
index.load_from_url("https://www.gutenberg.org/files/53669/53669-0.txt") # Portable Flame Thrower

2025-03-11 00:07:10,918 - INFO - GutenbergToBM25LlamaIndex initialized with Anthropic Haiku model
2025-03-11 00:07:10,920 - INFO - Loading https://www.gutenberg.org/files/53669/53669-0.txt from cache
2025-03-11 00:07:10,930 - INFO - Cleaned Gutenberg text: removed 50 chars from start, 49 chars from end
2025-03-11 00:07:11,525 - INFO - Successfully loaded text from https://www.gutenberg.org/files/53669/53669-0.txt -- 1369 nodes created.


## Step 2: Retrieve nodes that match query

In [7]:
retriever = BM25Retriever.from_defaults(
    docstore=index.get_indexed_documents(),
    similarity_top_k=5)

2025-03-11 00:07:11,657 - DEBUG - Building index from IDs objects


In [8]:
from llama_index.core.response.notebook_utils import display_source_node
retrieved_nodes = retriever.retrieve("What should I do if the diaphragm is ruptured?")
for node in retrieved_nodes:
    display_source_node(node, 1024)

**Node ID:** d6e235d3-6f40-4122-b5bb-96d532612423<br>**Similarity:** 4.394947528839111<br>**Text:** Remove deflector
tube from head (using hand, not wrench). Inspect to see if diaphragm
is intact. If diaphragm is ruptured, replace the safety head with an
unbroken head.<br>

**Node ID:** f110aa83-8e72-46ac-b87e-fd8d0347ac8b<br>**Similarity:** 3.875696897506714<br>**Text:** If diaphragm is ruptured, replace the safety head with an
unbroken head. (Par 69 b, c) Reassemble plug, head, and deflector tube
in left fuel tank.<br>

**Node ID:** 47042443-c65f-4a51-9e54-37b80ea77176<br>**Similarity:** 3.2352800369262695<br>**Text:** (3) Unscrew diaphragm cap and pull out washer, support, and
  valve-diaphragm assembly. To prevent loss of valve-needle adjustment
  (Fig 54), do not disturb position of yoke block by turning the needle.<br>

**Node ID:** c6183220-083f-492e-8ade-a044cf6508c2<br>**Similarity:** 2.645098924636841<br>**Text:** 46_f_
    thickener,                                              46_e_

  Diaphragm,                                                   75

  Diaphragm cap,                                               75

  Diaphragm support,                                           75

  Diaphragm, valve, assembly,  10_b_,<br>

**Node ID:** 64705a2f-a69f-4418-b4ef-30ad8ff669ec<br>**Similarity:** 2.6266260147094727<br>**Text:** (Fig 52) Screw on the diaphragm cap by hand. Do not use a wrench.
  Install valve grip. (Par 74 _c_)

  (4) Place valve spring over end of needle and install spring
  retainer.<br>

## Step 3: Generate using these nodes

In [9]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.anthropic import Anthropic

llm = Anthropic(
    model="claude-3-7-sonnet-latest",
    api_key=os.environ['ANTHROPIC_API_KEY'],
    temperature=0.2
)

In [10]:
from llama_index.core.llms import ChatMessage
messages = [
    ChatMessage(
        role="system", content="You are a mechanic. Use the information from the manual to answer the given query."
    )
]
messages += [
    ChatMessage(role="system", content=node.text) for node in retrieved_nodes
]
messages += [
    ChatMessage(role="user", content="What should I do if the diaphragm is ruptured?")
]
response = llm.chat(messages)
print(response)

2025-03-11 00:07:14,009 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


assistant: If the diaphragm is ruptured, you should replace the entire safety head with an unbroken head. Do not attempt to replace just the diaphragm - the manual specifically instructs to replace the complete safety head assembly when a ruptured diaphragm is found.


## Llama Query engine to simplify Step 3

In [11]:
query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever, llm=llm
)

response = query_engine.query("What should I do if the diaphragm is ruptured?")
response = {
    "answer": str(response),
    "source_nodes": response.source_nodes
}
print(response['answer'])

2025-03-11 00:07:15,626 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


If the diaphragm is ruptured, you should replace the safety head with an unbroken head. After replacement, you'll need to reassemble the plug, head, and deflector tube in the left fuel tank.


In [12]:
for node in response['source_nodes']:
    print(node)

Node ID: d6e235d3-6f40-4122-b5bb-96d532612423
Text: Remove deflector tube from head (using hand, not wrench).
Inspect to see if diaphragm is intact. If diaphragm is ruptured,
replace the safety head with an unbroken head.
Score:  4.395

Node ID: f110aa83-8e72-46ac-b87e-fd8d0347ac8b
Text: If diaphragm is ruptured, replace the safety head with an
unbroken head. (Par 69 b, c) Reassemble plug, head, and deflector tube
in left fuel tank.
Score:  3.876

Node ID: 47042443-c65f-4a51-9e54-37b80ea77176
Text: (3) Unscrew diaphragm cap and pull out washer, support, and
valve-diaphragm assembly. To prevent loss of valve-needle adjustment
(Fig 54), do not disturb position of yoke block by turning the needle.
Score:  3.235

Node ID: c6183220-083f-492e-8ade-a044cf6508c2
Text: 46_f_     thickener,
46_e_    Diaphragm,
75    Diaphragm cap,                                               75
Diaphragm support,                                           75
Diaphragm, valve, assembly,  10_b_,
Score:  2.645

Nod

## Limitation 1: Semantic Understanding

In [13]:
response = query_engine.query("What should I do if the diaphragm is broken?")
response = {
    "answer": str(response),
    "source_nodes": response.source_nodes
}
print(response['answer'])

2025-03-11 00:46:05,237 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


If the diaphragm is broken, you should replace it. The proper procedure would involve unscrewing the diaphragm cap and removing the washer, support, and valve-diaphragm assembly. When doing this, it's important not to disturb the position of the yoke block by turning the needle, as this would affect the valve-needle adjustment. After replacing the broken diaphragm, you would need to reassemble by screwing on the diaphragm cap by hand (not using a wrench), and then install the valve grip. You would also need to place the valve spring over the end of the needle and install the spring retainer.


In [14]:
retrieved_nodes = retriever.retrieve("What should I do if the diaphragm is broken?")
for node in retrieved_nodes:
    display_source_node(node, 1024)

**Node ID:** 47042443-c65f-4a51-9e54-37b80ea77176<br>**Similarity:** 3.2352800369262695<br>**Text:** (3) Unscrew diaphragm cap and pull out washer, support, and
  valve-diaphragm assembly. To prevent loss of valve-needle adjustment
  (Fig 54), do not disturb position of yoke block by turning the needle.<br>

**Node ID:** cac476f8-f3dc-4b8c-8a54-f26d377094b8<br>**Similarity:** 3.1163110733032227<br>**Text:** _e. Thickener._ Cans of thickener should be broken open. Contents
should be thrown into a fire or into a body of water.

_f. Ignition cylinders._ Burn to destroy.<br>

**Node ID:** ae7b7a6f-26c5-478e-9fa1-2af021feb38a<br>**Similarity:** 2.651236057281494<br>**Text:** (Par 49)

  (2) _Spring-case assembly._ If outer case rotates and inner case does
  not, and no spring action occurs, spring is broken and spring case
  should be replaced as a unit.<br>

**Node ID:** c6183220-083f-492e-8ade-a044cf6508c2<br>**Similarity:** 2.645098924636841<br>**Text:** 46_f_
    thickener,                                              46_e_

  Diaphragm,                                                   75

  Diaphragm cap,                                               75

  Diaphragm support,                                           75

  Diaphragm, valve, assembly,  10_b_,<br>

**Node ID:** 64705a2f-a69f-4418-b4ef-30ad8ff669ec<br>**Similarity:** 2.6266260147094727<br>**Text:** (Fig 52) Screw on the diaphragm cap by hand. Do not use a wrench.
  Install valve grip. (Par 74 _c_)

  (4) Place valve spring over end of needle and install spring
  retainer.<br>

## Limitation 2: Chunk size

In [15]:
def get_response(chunk_size: int) -> str:
    index = GutenbergToBM25LlamaIndex(chunk_size=chunk_size, chunk_overlap=chunk_size//10)
    index.load_from_url("https://www.gutenberg.org/files/53669/53669-0.txt") # Portable Flame Thrower
    retriever = BM25Retriever.from_defaults(
        docstore=index.get_indexed_documents(),
        similarity_top_k=5)
    query_engine = RetrieverQueryEngine.from_args(
        retriever=retriever, llm=llm
    )
    response = query_engine.query("What should I do if the diaphragm is ruptured?")
    response = {
        "answer": str(response),
        "source_nodes": response.source_nodes
    }
    return response['answer']

get_response(100)

2025-03-11 00:52:17,874 - INFO - GutenbergToBM25LlamaIndex initialized with Anthropic Haiku model
2025-03-11 00:52:17,876 - INFO - Loading https://www.gutenberg.org/files/53669/53669-0.txt from cache
2025-03-11 00:52:17,884 - INFO - Cleaned Gutenberg text: removed 50 chars from start, 49 chars from end
2025-03-11 00:52:18,539 - INFO - Successfully loaded text from https://www.gutenberg.org/files/53669/53669-0.txt -- 1208 nodes created.
2025-03-11 00:52:18,660 - DEBUG - Building index from IDs objects
2025-03-11 00:52:22,042 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


'If the diaphragm is ruptured, you should replace the safety head with an unbroken head. Additionally, if you notice any tears, separation, or leaks at the diaphragm, you should replace the entire valve-diaphragm assembly.\n\nWhen working with the diaphragm, remember to handle it carefully. When reassembling, screw on the diaphragm cap by hand without using a wrench. Also, when removing the diaphragm cap and pulling out the washer, support, and valve-diaphragm assembly, be careful not to disturb the position of the yoke block by turning the needle, as this would affect the valve-needle adjustment.'

In [16]:
get_response(200)

2025-03-11 00:52:35,827 - INFO - GutenbergToBM25LlamaIndex initialized with Anthropic Haiku model
2025-03-11 00:52:35,829 - INFO - Loading https://www.gutenberg.org/files/53669/53669-0.txt from cache
2025-03-11 00:52:35,837 - INFO - Cleaned Gutenberg text: removed 50 chars from start, 49 chars from end
2025-03-11 00:52:36,042 - INFO - Successfully loaded text from https://www.gutenberg.org/files/53669/53669-0.txt -- 376 nodes created.
2025-03-11 00:52:36,092 - DEBUG - Building index from IDs objects
2025-03-11 00:52:38,511 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


'If the diaphragm is ruptured, you should replace the safety head with an unbroken head. This is necessary when inspecting the deflector tube from the head. Before doing this, you would need to turn the tank upside down and remove any broken rod or rod and chain that may have fallen in.'

In [17]:
get_response(500)

2025-03-11 00:53:07,707 - INFO - GutenbergToBM25LlamaIndex initialized with Anthropic Haiku model
2025-03-11 00:53:07,709 - INFO - Loading https://www.gutenberg.org/files/53669/53669-0.txt from cache
2025-03-11 00:53:07,716 - INFO - Cleaned Gutenberg text: removed 50 chars from start, 49 chars from end
2025-03-11 00:53:07,903 - INFO - Successfully loaded text from https://www.gutenberg.org/files/53669/53669-0.txt -- 124 nodes created.
2025-03-11 00:53:07,938 - DEBUG - Building index from IDs objects
2025-03-11 00:53:12,028 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


"If you find that the diaphragm is ruptured, you should replace the safety head with an unbroken head. This is important maintenance for proper functioning of the equipment. After replacement, you'll need to reassemble the plug, head, and deflector tube in the left fuel tank. When reinstalling, make sure the tube faces to the rear at a 45-degree angle to the operator's left shoulder. Remember to screw in the deflector tube by hand only (do not use a wrench on it), and then tighten the lock nut with a wrench."